In [ ]:
import os
import pandas as pd
import joblib
import time
import json
from xgboost import XGBClassifier

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Paths
SPLITS_DIR = "/content/drive/My Drive/FDM Project Files/splits"
XGB_DIR = "/content/drive/My Drive/FDM Project Files/models/xgboost"
os.makedirs(XGB_DIR, exist_ok=True)

# Load train data
X_train = pd.read_csv(os.path.join(SPLITS_DIR, "X_train.csv"))
y_train = pd.read_csv(os.path.join(SPLITS_DIR, "y_train.csv")).squeeze()

In [ ]:
# Define model
xgb = XGBClassifier(
    n_estimators=300,
    learning_rate=0.1,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    use_label_encoder=False,
    eval_metric="logloss"
)

In [ ]:
# Encode target variable
y_train = y_train.map({'Left': 1, 'Stayed': 0})

In [ ]:
# One-hot encode the 'Overtime' column in X_train
X_train = pd.get_dummies(X_train, columns=['Overtime'], drop_first=True)

# Save column order for consistency in evaluation
columns_path = os.path.join(XGB_DIR, "xgb_columns.json")
with open(columns_path, "w") as f:
    json.dump(list(X_train.columns), f)

# Train and time it
start_train = time.time()
xgb.fit(X_train, y_train)
end_train = time.time()
train_time = round(end_train - start_train, 3)

print(f" XGBoost trained successfully in {train_time} seconds")

/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [11:54:13] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


 XGBoost trained successfully in 1.729 seconds


In [ ]:
# Save model
model_path = os.path.join(XGB_DIR, "xgb_model.pkl")
joblib.dump(xgb, model_path)
print(" Model saved at:", model_path)

# Save training metadata
meta = {"Train_Time_sec": train_time}
with open(os.path.join(XGB_DIR, "xgb_meta.json"), "w") as f:
    json.dump(meta, f)
print(" Training metadata saved")

 Model saved at: /content/drive/My Drive/FDM Project Files/models/xgboost/xgb_model.pkl
 Training metadata saved
